In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import boto3

# Create an Athena client
athena_client = boto3.client('athena')

In [9]:
# Set the name of the S3 bucket where the query results will be stored
s3_bucket = 'bw-query-results'

# Set the name of the database and table you want to query
database_name = 'sports-analytics'
table_name = 'ss-defense'

# Set the query string
query_string = f"SELECT * FROM \"{database_name}\".\"{table_name}\" WHERE \"team\" = 'USU'"
print(query_string)

# Execute the query
response = athena_client.start_query_execution(
    QueryString=query_string,
    ResultConfiguration={
        'OutputLocation': f's3://{s3_bucket}/postgres-py/'
    }
)

# Get the query execution ID
query_execution_id = response['QueryExecutionId']
print(query_execution_id)

# Wait for the query to complete
query_status = 'QUEUED'
while query_status == 'QUEUED' or query_status == 'RUNNING':
    response = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
    query_status = response['QueryExecution']['Status']['State']
    print(query_status)

# TODO throw error if status <> SUCCEEDED
# Get the query results
response = athena_client.get_query_results(QueryExecutionId=query_execution_id)


SELECT * FROM "sports-analytics"."ss-defense" WHERE "team" = 'USU'
412641df-c906-4000-857f-aced13fdd4fa
QUEUED
QUEUED
QUEUED
QUEUED
QUEUED
QUEUED
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
SUCCEEDED


In [31]:
# Get the list of rows from the query results
rows = response['ResultSet']['Rows']

# Print the rows
print(len(rows))
for row in rows:
    print(row)
    print('')
    print(row['Data'][0]['VarCharValue'])


80
{'Data': [{'VarCharValue': 'player'}, {'VarCharValue': 'team'}, {'VarCharValue': 'int'}, {'VarCharValue': 'yds'}, {'VarCharValue': 'avg'}, {'VarCharValue': 'lg'}, {'VarCharValue': 'td'}, {'VarCharValue': 'solo'}, {'VarCharValue': 'ast'}, {'VarCharValue': 'tot'}, {'VarCharValue': 'sack'}, {'VarCharValue': 'ydsl'}, {'VarCharValue': 'stat type'}, {'VarCharValue': 'conf_syscol'}, {'VarCharValue': 'year_syscol'}]}
/n
player
{'Data': [{'VarCharValue': 'Brian Suite'}, {'VarCharValue': 'USU'}, {'VarCharValue': '5'}, {'VarCharValue': '0'}, {'VarCharValue': '0.0'}, {'VarCharValue': '0'}, {'VarCharValue': '0'}, {'VarCharValue': '37'}, {'VarCharValue': '40'}, {'VarCharValue': '77'}, {'VarCharValue': '0.0'}, {'VarCharValue': '0.0'}, {'VarCharValue': 'Defense'}, {'VarCharValue': 'allfbs'}, {'VarCharValue': '2013'}]}
/n
Brian Suite
{'Data': [{'VarCharValue': 'Nevin Lawson'}, {'VarCharValue': 'USU'}, {'VarCharValue': '4'}, {'VarCharValue': '65'}, {'VarCharValue': '16.25'}, {'VarCharValue': '65t'}, 

In [30]:
print(rows[1]['Data'][0]['VarCharValue'])

Brian Suite


In [39]:
from enum import Enum

In [40]:
class DefenseColumn(Enum):
    PLAYER = 0
    TEAM = 1
    INTERCEPTIONS = 2
    YARDS = 3
    AVG_INT_RETURN_YARDS = 4
    LONGEST_INT = 5
    TOUCHDOWNS = 6
    SOLO_TACKLES = 7
    ASSISTED_TACKLES = 8
    TOTAL_TACKLES = 9
    SACKS = 10
    SACK_YARDS_LOST = 11
    STAT_TYPE = 12
    CONFERENCE = 13
    YEAR = 14

In [41]:
print(DefenseColumn.SACKS.value)

10


In [43]:
team = row['Data'][DefenseColumn.CONFERENCE.value]['VarCharValue']
print(team)

mountainwest


In [ ]:
sys.path.append('../util/')